In [3]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from transformers import CamembertConfig, CamembertModel, AutoTokenizer, CamembertTokenizer, CamembertForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
import functools
from tqdm import tqdm
import gc

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
#Load 20 minutes articles
df_articles = pd.read_parquet('../newspaper_2.parquet')

In [3]:
df_articles = df_articles.sort_values("article_date")
df_autres = df_articles[(df_articles.category_id != 'planete') & (df_articles.category_id != 'sport') & (df_articles.category_id != 'economie') & (df_articles.category_id != 'arts-stars') & (df_articles.category_id != 'high-tech') & (df_articles.category_id != 'politique')]
df_autres = df_autres[(df_autres.body != '') & (df_autres.title != '')]
df_autres["category_id"] = "autres"
df_articles = df_articles[(df_articles.category_id == 'planete') | (df_articles.category_id == 'sport')| (df_articles.category_id == 'economie')| (df_articles.category_id == 'arts-stars')| (df_articles.category_id == 'high-tech')| (df_articles.category_id == 'politique')]
df_articles = df_articles[(df_articles.body != '') & (df_articles.title != '')]
df_articles.groupby("category_id").count()[['title']]

,title
category_id,
arts-stars,28294
economie,41992
high-tech,13565
planete,11055
politique,16601
sport,71847


In [4]:
dict_labels = {'planete': 0, 'sport': 1, 'economie': 2, 'arts-stars': 3, 'high-tech': 4, 'politique': 5, 'autres': 6}
p_train, p_test = df_articles[(df_articles.category_id == 'planete')].iloc[:7000],  df_articles[(df_articles.category_id == 'planete')].iloc[7000:8000]
s_train, s_test = df_articles[(df_articles.category_id == 'sport')].iloc[:7000],  df_articles[(df_articles.category_id == 'sport')].iloc[7000:8000]
e_train, e_test = df_articles[(df_articles.category_id == 'economie')].iloc[:7000],  df_articles[(df_articles.category_id == 'economie')].iloc[7000:8000]
sc_train, sc_test = df_articles[(df_articles.category_id == 'arts-stars')].iloc[:7000],  df_articles[(df_articles.category_id == 'arts-stars')].iloc[7000:8000]
h_train, h_test = df_articles[(df_articles.category_id == 'high-tech')].iloc[:7000],  df_articles[(df_articles.category_id == 'high-tech')].iloc[7000:8000]
po_train, po_test = df_articles[(df_articles.category_id == 'politique')].iloc[:7000],  df_articles[(df_articles.category_id == 'politique')].iloc[7000:8000]
a_train, a_test = train_test_split(df_autres, test_size=14000, train_size=5000, random_state=42)
train_dataset = pd.concat([p_train, s_train, e_train, sc_train, h_train, po_train, a_train])[['title', 'category_id']]
train_dataset['label'] = train_dataset.apply(lambda x: dict_labels[x['category_id']], axis=1)
test_dataset = pd.concat([p_test, s_test, e_test, sc_test, h_test, po_test, a_test])[['title', 'category_id']]
test_dataset['label'] = test_dataset.apply(lambda x: dict_labels[x['category_id']], axis=1)

In [5]:
# Set gloabal parameters and tokenizer
MAX_LEN = 64
batch_size = 32
tokenizer = CamembertTokenizer.from_pretrained('camembert-base',do_lower_case=True)

In [6]:
### Creation of the test dataset ###
# Creates list of texts and labels
text = test_dataset['title'].to_list()
labels = test_dataset['label'].to_list()

#user tokenizer to convert sentences into tokenizer
input_ids  = tokenizer(text, max_length=MAX_LEN, padding='longest', truncation=True).input_ids

# Create attention masks
attention_masks = []
# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
    seq_mask = [float(i!=1) for i in seq]  
    attention_masks.append(seq_mask)

In [7]:
# transfrom to tensor format
test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)

# create dataloader
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [8]:
# Creates list of texts and labels
text = train_dataset['title'].to_list()
labels = train_dataset['label'].to_list()

#user tokenizer to convert sentences into tokenizer
input_ids  = tokenizer(text, max_length=MAX_LEN, padding='longest', truncation=True).input_ids

# Create attention masks
attention_masks = []
# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
    seq_mask = [float(i!=1) for i in seq]  
    attention_masks.append(seq_mask)

In [9]:
# transfrom to tensor format
train_inputs = torch.tensor(input_ids)
train_labels = torch.tensor(labels)
train_masks = torch.tensor(attention_masks)

# create dataloader
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

In [4]:
# Load pretained model
model = CamembertForSequenceClassification.from_pretrained("camembert-base", num_labels=7).to(device)

Some weights of the model checkpoint at camembert-base were not used when initializing CamembertForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.out_proj.bias

In [11]:
# function to compute accuracy

def compute_accuracy(test_dataloader, model):
    total_true = 0
    total_size = 0
    with torch.no_grad():
        for batch in tqdm(test_dataloader):
            t_data = batch[0].to(device)
            t_mask = batch[1].to(device)
            y = model(t_data,attention_mask=t_mask).logits
            result = torch.argmax(y, dim=-1).cpu().detach().numpy()
            nb_true, size = np.sum(result == np.array(batch[2])), len(result)
            total_true += nb_true
            total_size += size
            t_data.cpu()
            t_mask.cpu()
            del batch
            del y
            del t_data
            del t_mask
            gc.collect()
        
    accuracy = total_true / total_size
    return accuracy

In [ ]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

In [12]:
# extract parameters to optimize
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

opt = torch.optim.Adam(optimizer_grouped_parameters, lr=2e-5, eps=10e-8)


def train(classifier, data, epochs=5, log_interval=50):
    best_acc = 0
    losses = []
    for epoch in range(epochs):
        total_loss = 0
        for idx, batch in enumerate(data):
            # Unpack the inputs from our dataloader
            b_input_ids, b_input_mask, b_labels = batch[0].to(device), batch[1].to(device), batch[2].to(device)
            # Train model on batch
            opt.zero_grad()
            y_pred = classifier(b_input_ids, attention_mask=b_input_mask, labels=b_labels)
            loss = y_pred[0]
            loss.backward()
            opt.step()
            total_loss += loss.item()
            # delete variables to keep memory on gpu
            loss.detach().cpu()
            b_input_ids.cpu()
            b_input_mask.cpu()
            b_labels.cpu()
            del(y_pred)
            del(loss)
            del(batch)
            del(b_input_ids, b_input_mask, b_labels)
            gc.collect()
            if idx % log_interval == 0 and idx > 0:
                cur_loss = total_loss / log_interval
                print(
                    "| epoch {:3d} | {:5d}/{:5d} steps | "
                    "loss {:5.5f}".format(
                        epoch, idx, len(data), cur_loss,
                    )
                )
                losses.append(cur_loss)
                total_loss = 0
        accuracy = compute_accuracy(test_dataloader, model)
        print("Test accuracy : {:1.3f}".format(accuracy))
        # Save model if better
        if accuracy > best_acc:
            torch.save(model.state_dict(), '../models/camembert_title_14K.pt')
            best_acc = accuracy
    return classifier

In [13]:
model = train(model, train_dataloader, 2)

| epoch   0 |    50/ 1469 steps | loss 1.81394
| epoch   0 |   100/ 1469 steps | loss 1.25588
| epoch   0 |   150/ 1469 steps | loss 0.89543
| epoch   0 |   200/ 1469 steps | loss 0.78953
| epoch   0 |   250/ 1469 steps | loss 0.67403
| epoch   0 |   300/ 1469 steps | loss 0.62081
| epoch   0 |   350/ 1469 steps | loss 0.58372
| epoch   0 |   400/ 1469 steps | loss 0.56182
| epoch   0 |   450/ 1469 steps | loss 0.47667
| epoch   0 |   500/ 1469 steps | loss 0.50559
| epoch   0 |   550/ 1469 steps | loss 0.49567
| epoch   0 |   600/ 1469 steps | loss 0.44547
| epoch   0 |   650/ 1469 steps | loss 0.45166
| epoch   0 |   700/ 1469 steps | loss 0.46312
| epoch   0 |   750/ 1469 steps | loss 0.49224
| epoch   0 |   800/ 1469 steps | loss 0.45498
| epoch   0 |   850/ 1469 steps | loss 0.46739
| epoch   0 |   900/ 1469 steps | loss 0.42009
| epoch   0 |   950/ 1469 steps | loss 0.47173
| epoch   0 |  1000/ 1469 steps | loss 0.44227
| epoch   0 |  1050/ 1469 steps | loss 0.45995
| epoch   0 |

100%|██████████| 625/625 [02:52<00:00,  3.63it/s]


Test accuracy : 0.671
| epoch   1 |    50/ 1469 steps | loss 0.33204
| epoch   1 |   100/ 1469 steps | loss 0.24871
| epoch   1 |   150/ 1469 steps | loss 0.26962
| epoch   1 |   200/ 1469 steps | loss 0.29491
| epoch   1 |   250/ 1469 steps | loss 0.25126
| epoch   1 |   300/ 1469 steps | loss 0.28984
| epoch   1 |   350/ 1469 steps | loss 0.29596
| epoch   1 |   400/ 1469 steps | loss 0.27086
| epoch   1 |   450/ 1469 steps | loss 0.27196
| epoch   1 |   500/ 1469 steps | loss 0.27721
| epoch   1 |   550/ 1469 steps | loss 0.26761
| epoch   1 |   600/ 1469 steps | loss 0.25930
| epoch   1 |   650/ 1469 steps | loss 0.26737
| epoch   1 |   700/ 1469 steps | loss 0.29556
| epoch   1 |   750/ 1469 steps | loss 0.29903
| epoch   1 |   800/ 1469 steps | loss 0.27771
| epoch   1 |   850/ 1469 steps | loss 0.29790
| epoch   1 |   900/ 1469 steps | loss 0.27438
| epoch   1 |   950/ 1469 steps | loss 0.25406
| epoch   1 |  1000/ 1469 steps | loss 0.22844
| epoch   1 |  1050/ 1469 steps | loss

100%|██████████| 625/625 [02:49<00:00,  3.69it/s]


Test accuracy : 0.704


In [14]:
torch.save(model.state_dict(), '../models/camembert_title_7K.pt')